<h1>Problem-Security Correlation Analysis</h1>
In this problem you'll get some practice getting and combining data from the St. Louis Federal Reserve (FRED). Get the following data from FRED (01/01/2010 to 12/31/2022):

<pre>
"TB3MS" #3 month t-bill market yield 
"DGS10" #10 year constant maturity government bond market yield
"NB000334Q" #Real GDP index quarterly (index = 100 at 2012)
"CPIAUCSL" #Consumer price index for all urban consumers seasonally adjusted
</pre>

Since these data items have different frequencies (some are daily, some monthly, some quarterly), make separate data reader calls for each. For GDP and the CPI, use percent changes quarter over quarter rather than the absolute values

Then, as a proxy for the stock market, get data for the ticker SPY,the S&P ETF, from tiingo. You will need to create an account and get an API Key (https://www.tiingo.com/). Use the adjusted close. Resample the data to the business quarter and calculate a quarter over quarter percent change.

Align all the data to the end of the business quarter (i.e., use the value on the last day of each quarter).

For the ETF, calculate one day percent changes and shift the data back by one quarter (we're interested in the correlation between macroeconomic data in one quarter and the performance of the S&P in the next quarter). For example, if the percentage change on 3/31 is 5% and on 6/30 is 2.5%, we want to align the percent change on 6/30 with the the macroeconomic data as of 3/31. So we need to replace the data on 3/31 by 2.5%

Using the pandas join function, join all the data into one dataframe with the quarter end date as the index

Generate the correlation matrix. This is what you should get:

<pre>
	TB3MS	DGS10	NB000334Q	CPIAUCSL	SPY
TB3MS	1.000000	0.409817	0.070356	-0.001078	-0.078983
DGS10	0.409817	1.000000	0.060454	0.038290	-0.329595
NB000334Q	0.070356	0.060454	1.000000	0.089625	-0.009725
CPIAUCSL	-0.001078	0.038290	0.089625	1.000000	-0.393822
SPY	-0.078983	-0.329595	-0.009725	-0.393822	1.000000

</pre>

<h3>Notes:</h3>

1. In the shift function, positive numbers will shift forward while negative numbers will shift backward
2. tiingo returns datetime index values while fred returns date index values. You can convert datetime to date using:

    df.index = df.index.date
    
where spy is the dateframe with datetime values as index
    
3. To rename a column, use df.rename(columns={"old_name":"new_name"})

In [1]:
#Your code goes here

from pandas_datareader import data as pdr
import datetime
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2022, 12, 31)

#get data from FRED
tbill = pdr.DataReader("TB3MS", "fred", start, end)
bond = pdr.DataReader("DGS10", "fred", start, end)
gdp = pdr.DataReader("NB000334Q", "fred", start, end)
cpi = pdr.DataReader("CPIAUCSL", "fred", start, end)

#get data from tiingo
TIINGO_API_KEY = "539c2e5c0b58b8b97c26675c06aaf043e0dc5d76"
spy = pdr.DataReader(["SPY"], "tiingo", start, end, api_key = TIINGO_API_KEY).loc["SPY"]

/var/folders/94/_mqchpnd0kv5pd779q9n49jh0000gn/T/ipykernel_13080/2657414343.py:16: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  spy = pdr.DataReader(["SPY"], "tiingo", start, end, api_key = TIINGO_API_KEY).loc["SPY"]


In [2]:
#for GDP and CPI: percent changes quarter over quarter
gdp_pct = gdp.pct_change() 
cpi_pct = cpi.pct_change(3) 

In [4]:
import pandas as pd
#for S&P ETF: use asjClose, resample the data to business quarter
##and calculate a quarter over quarter percent change
spy_pct = spy["adjClose"].resample("BQ").last().pct_change()

#align date index
spy_pct.index = pd.DatetimeIndex(spy_pct.index) 
spy_pct.index = spy_pct.index.tz_convert(None)

##Notes: since "spy_pct.index = spy_pct.index.date" will derive a warning in join function: 
###"Comparison of Timestamp with datetime.date is deprecated",
###I change the datetime-to-date conversion method

In [5]:
#for S&P ETF: shift the data back back one quarter 
##and align the object type and column name 
spy_pct = spy_pct.shift(-1).to_frame().rename(columns={"adjClose":"SPY"}) 

In [6]:
#align all the data to the end of the business quarter
tbill = tbill.resample("BQ").last()
bond = bond.resample("BQ").last()
gdp_pct = gdp_pct.resample("BQ").last()
cpi_pct = cpi_pct.resample("BQ").last()

In [7]:
#join all the data into one dataframe 
data = tbill.join([bond, gdp_pct, cpi_pct, spy_pct])

In [8]:
#generate the correlation matrix
corr_matrix = data.corr()
print(corr_matrix)

              TB3MS     DGS10  NB000334Q  CPIAUCSL       SPY
TB3MS      1.000000  0.409817   0.063098  0.049515 -0.078983
DGS10      0.409817  1.000000   0.056068  0.060827 -0.329595
NB000334Q  0.063098  0.056068   1.000000  0.162265 -0.009725
CPIAUCSL   0.049515  0.060827   0.162265  1.000000 -0.398769
SPY       -0.078983 -0.329595  -0.009725 -0.398769  1.000000
